In [ ]:
import numpy as np
import pandas as pd
# import tensorflow as tf
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
import numpy as np

# Load datasets
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Drop unnecessary columns
train_df = train_df.drop(columns=['trip_ID', 'days_before_booked', 'weather_at_arrival'])
test_df = test_df.drop(columns=['trip_ID', 'days_before_booked', 'weather_at_arrival'])

# Handle missing values
for column in train_df.columns:
    if train_df[column].isnull().any():
        if train_df[column].dtype == 'object':
            train_df[column] = train_df[column].fillna(train_df[column].mode()[0])
        else:
            train_df[column] = train_df[column].fillna(train_df[column].median())

# Apply the same handling to test data
for column in test_df.columns:
    if test_df[column].isnull().any():
        if test_df[column].dtype == 'object':
            test_df[column] = test_df[column].fillna(test_df[column].mode()[0])
        else:
            test_df[column] = test_df[column].fillna(test_df[column].median())

# Encode categorical variables
train_df = pd.get_dummies(train_df, drop_first=True)
test_df = pd.get_dummies(test_df, drop_first=True)

# Align train and test datasets
train_df, test_df = train_df.align(test_df, join='left', axis=1, fill_value=0)

# Separate features and target variable
X = train_df.drop(columns=['category'])
y = train_df['category']

# Align train and test datasets on features
X, test_df = X.align(test_df, join='left', axis=1, fill_value=0)
print(f"Training features: {X.shape}")
print(f"Test features: {test_df.shape}")

# Scale numerical features
scaler = StandardScaler()
X = scaler.fit_transform(X)
test_df = scaler.transform(test_df)

# Split data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

# Train SVM
svc = SVC(kernel='rbf', C=1, gamma='scale', probability=True, random_state=42)
svc.fit(X_train, y_train)

# Evaluate SVM
y_pred = svc.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f"SVM Accuracy: {accuracy:.4f}")
print("Classification Report:")
print(classification_report(y_val, y_pred))



# Make predictions on test data
test_df_og = pd.read_csv('test.csv')  # Reload for trip_ID
predictions = svc.predict(test_df)

# Save submission
submission = pd.DataFrame({'trip_ID': test_df_og['trip_ID'], 'category': predictions})
submission.to_csv('submission_svm.csv', index=False)
print("SVM predictions saved to submission_svm.csv")


Training features: (12654, 172)
Test features: (5852, 172)
